In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/small_language_model

/content/gdrive/MyDrive/small_language_model


In [3]:
#!git clone https://github.com/patricknormile/small-language-model.git

In [4]:
%cd small-language-model/

/content/gdrive/MyDrive/small_language_model/small-language-model


In [5]:
!ls

artifacts	       LICENSE	    __pycache__       results
create_dataset.py      non_working  README.md	      slm-env
create_environment.sh  outputs	    requirements.txt  training_notebook.ipynb


In [6]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 15.4 MB/s eta 0:00:00
  Using cached 

In [7]:
!pip install accelerate -U

In [8]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from datasets import Dataset
from torch.utils.data import dataset
import torch.utils.data as data
import mlflow
import torch
from transformers import Trainer, TrainingArguments
from transformer_lens import utils, HookedTransformer
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
import mlflow
from transformers.integrations import MLflowCallback

from create_dataset import make_rows_from_chat

file_name = "artifacts/input_text.txt"

with open(file_name, "r") as file_read:
    chat = file_read.read()

chat_rows = make_rows_from_chat(chat)

In [9]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
def find_missing_tokens(data, tokenizer):
    missing_tokens = set()
    for row in data:
        tokens = re.findall(r'\b\w+\b', row)
        for token in tokens:
            if tokenizer.convert_tokens_to_ids(token) == tokenizer.unk_token_id:
                missing_tokens.add(token)
    return list(missing_tokens)

data = chat_rows[-3000:]
# Find missing tokens in your data
missing_tokens = find_missing_tokens(data, tokenizer)
print("Missing tokens:", missing_tokens)

# Add missing tokens to the tokenizer
new_tokens = missing_tokens
num_added_tokens = tokenizer.add_tokens(new_tokens)
print(f"Added {num_added_tokens} tokens.")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
def make_generator(data, tokenizer) :
    for x in data :
        tokens = tokenizer(x, truncation=True, padding=True)['input_ids']
        for i in range(1,len(tokens), 1) :
            x, y = tokens[:i], tokens[i]
            yield {"text":tokenizer.decode(x), "target":tokenizer.decode(y)}
ds = Dataset.from_generator(make_generator,gen_kwargs={'data':data,
'tokenizer':tokenizer})
ds = ds.with_format("torch")
ds


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Missing tokens: ['definitely', 'Fabian', 'Redgarden', 'Ya', 'volunteer', 'Doesn', 'itself', 'blazing', '1179268399537791', 'comfortable', '16195694062', 'whom', 'closeby', 'mapper', 'Oops', 'drove', 'CDHE', 'freerider', 'Anthill', 'slam', '2023', 'dangles', 'protective', 'exciting', 'Ergh', 'Drone', 'OPTION', 'thoughts', 'decide', 'EACH', 'skills', 'plz', 'Lol', 'message_app', 'divergent', 'Autocorrect', 'buds', 'risking', 'Cheers', 'withdraw', 'implications', 'sessions', '105811510', 'conversation', 'kebob', 'separated', 'relax', 'abducted', 'interview', 'responses', 'entirely', 'leclerc', 'dying', 'consistent', 'newly', '6609', 'uniform', 'boco', '2Caps', 'GXBkOtqkmNXEwx8CBhhCFq', 'pregnancy', 'publishing', 'crushed', 'shame', '0021', 'privileges', 'competely', 'visit', 'hesitate', 'unwell', 'combo', 'sailing', 'animals', 'Likely', 'Lmk', 'stopped', 'enrolled', '20Money', 'gala', 'understanding', 'sake', 'impersonation', 'glider', '0674', 'calfs', 'Promaster', 'trick', 'regulars', 's

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text', 'target'],
    num_rows: 66311
})

In [10]:
ds.features

{'text': Value(dtype='string', id=None),
 'target': Value(dtype='string', id=None)}

In [11]:
# Initialize the model
model = GPT2LMHeadModel.from_pretrained('gpt2')
# Resize the model's token embeddings to match the new tokenizer size
model.resize_token_embeddings(len(tokenizer))

# Save the updated tokenizer
tokenizer.save_pretrained('outputs/gpt2_tokenizer')
for param in model.parameters():
    param.requires_grad = False

for param, name in zip(model.parameters(),model.named_parameters()) :
    if not re.match("^transformer.h.11.",name[0]) :
        continue
    param.requires_grad = True

input_text = "Hello, would you mind"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
def next_token(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(input_ids)

    # Extract logits for the last token
    logits = outputs.logits
    last_token_logits = logits[0, -1, :]

    # Apply softmax to get probabilities
    probabilities = torch.softmax(last_token_logits, dim=-1)

    # Find the predicted token id
    predicted_token_id = torch.argmax(probabilities).item()

    # Decode the predicted token to get the word
    predicted_token = tokenizer.decode([predicted_token_id])
    return predicted_token

print(input_text)
for i in range(15):
    add = next_token(input_text)
    input_text += add
    print(input_text)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Hello, would you mind
Hello, would you mindcertifiedhandtherapyservices
Hello, would you mindcertifiedhandtherapyservicescertifiedhandtherapyservices
Hello, would you mindcertifiedhandtherapyservicescertifiedhandtherapyservicescertifiedhandtherapyservices
Hello, would you mindcertifiedhandtherapyservicescertifiedhandtherapyservicescertifiedhandtherapyservicescertifiedhandtherapyservices
Hello, would you mindcertifiedhandtherapyservicescertifiedhandtherapyservicescertifiedhandtherapyservicescertifiedhandtherapyservicescertifiedhandtherapyservices
Hello, would you mindcertifiedhandtherapyservicescertifiedhandtherapyservicescertifiedhandtherapyservicescertifiedhandtherapyservicescertifiedhandtherapyservicescertifiedhandtherapyservices
Hello, would you mindcertifiedhandtherapyservicescertifiedhandtherapyservicescertifiedhandtherapyservicescertifiedhandtherapyservicescertifiedhandtherapyservicescertifiedhandtherapyservicescertifiedhandtherapyservices
Hello, would you mindcertifiedhandtherap

In [12]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [13]:
#print(train_ds[0])
#print(test_ds[0])


In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

def tokenize_function(examples):
    inputs = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)
    targets = tokenizer(examples['target'], truncation=True, padding='max_length', max_length=512)
    labels = targets['input_ids'].copy()
    for i in range(len(labels)):
        labels[i] = labels[i][1:] + [tokenizer.pad_token_id]
    inputs['labels'] = targets['input_ids']
    return inputs

tokenized_datasets = ds.map(tokenize_function, batched=True)
print("Sample before tensor conversion:")
print(tokenized_datasets[0])
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_datasets = tokenized_datasets.with_format("torch",device=device)
split_ds = tokenized_datasets.train_test_split(test_size=0.1)
train_ds = split_ds["train"]
test_ds = split_ds["test"]

print("Sample after tensor conversion:")
print(next(iter(train_ds)))

training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=32,  # Adjust batch size as needed
    save_steps=10_000,
    save_total_limit=2,
    learning_rate=0.001,
    logging_dir='./logs',
    logging_steps=200,
    report_to="none"  # Disable reporting to W&B
)

# Set up MLflow
mlflow.set_tracking_uri("./outputs")  # Replace with your tracking URI
mlflow.set_experiment("gpt2_fine_tuning")



Map:   0%|          | 0/66311 [00:00<?, ? examples/s]

Sample before tensor conversion:
{'text': 'Hey', 'target': ' all', 'input_ids': tensor([10814, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553,
        53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553,
        53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553,
        53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553,
        53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553,
        53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553,
        53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553,
        53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553,
        53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553,
        53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553,
        53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553,
        53553, 53553, 53553, 53553, 53553, 53553, 53553, 53553

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/mlflow/store/tracking/file_store.py", line 315, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/usr/local/lib/python3.10/dist-packages/mlflow/store/tracking/file_store.py", line 408, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/usr/local/lib/python3.10/dist-packages/mlflow/store/tracking/file_store.py", line 1339, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/usr/local/lib/python3.10/dist-packages/mlflow/store/tracking/file_store.py", line 1332, in _read_helper
    result = read_yaml(root, file_name)
  File "/usr/local/lib/python3.10/dist-packages/mlflow/utils/file_utils.py", line 312, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist.")
mlflow.exceptions.MissingConfigException: Yaml file './outputs/gpt2_tokenizer/meta.yaml' does

<Experiment: artifact_location='/content/gdrive/MyDrive/small_language_model/small-language-model/outputs/873641793641657509', creation_time=1717518904181, experiment_id='873641793641657509', last_update_time=1717518904181, lifecycle_stage='active', name='gpt2_fine_tuning', tags={}>

In [ ]:
# Initialize Trainer with MLflow callback
model.to(device)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    data_collator=data_collator,
    callbacks=[MLflowCallback()],
)

In [ ]:
# Start training
with mlflow.start_run():
    trainer.train()
    eval_metrics = trainer.evaluate(eval_dataset=test_ds)
    for key, value in eval_metrics.items():
        mlflow.log_metric(key, value)
    mlflow.pyfunc.log_model(model)

In [ ]:
# see how it changed
input_text = "Hello, would you mind"
print(input_text)
for i in range(15):
    add = next_token(input_text)
    input_text += add
    print(input_text)

In [ ]:
model.save_pretrained('outputs/saved_model')